In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
!pip install transformers==3.4
!pip install streamlit
!pip install pyngrok==4.2.2
!pip install pyspellchecker
!pip install plotty
import transformers
from spellchecker import SpellChecker
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
import time
import nltk
nltk.download('stopwords')
nltk.download('punkt')

      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.2.2-cp36-none-any.whl size=18025 sha256=4bd00c466067159fc379f02419031f70ebe920d7aeffe0899a25c2f5bc996486
  Stored in directory: /root/.cache/pip/wheels/9c/d3/b5/502fbef4b2782a6142bc00c62ccde76ee04c9148fb43b1b2cd
Successfully built pyngrok
     |████████████████████████████████| 2.5MB 4.1MB/s 
ERROR: Could not find a version that satisfies the requirement plotty (from versions: none)
ERROR: No matching distribution found for plotty
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
%%writefile main.py
import streamlit  as st
PAGE_CONFIG = {"page_title": "AI Answer Evaluation", "page_icon": "📚","layout": "centered"}
st.set_page_config(**PAGE_CONFIG)
import tensorflow as tf
import transformers
import numpy as np
import pandas as pd
import streamlit as st
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
import plotly.express as px
import matplotlib as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


 
#SideBar Content
st.markdown("""<style>.sidebar.sidebar-content{background-image: linear-gradient(rgb(54,112,130),rgb(54,112,130)); size=40px; }</style> """,
    unsafe_allow_html=True,)
#Selectbox
st.markdown("""<style>.sidebar .selectbox .selectbox-content {background-image: linear-gradient(rgb(54,112,130), rgb(54,112,130));color: white;size: 40px ;} </style>""",
    unsafe_allow_html=True,)

#Title
m_c_m = """<div style= padding: 10px;"><h1 style = "color: rgb(255,255,255); text-align: center; style = Times New Roman; font-size: 70x;">AI based Subjective Answer Evaluation</h1></div>"""
st.markdown(m_c_m, unsafe_allow_html=True)

#Coloring Text
colouringTool = '''<style>.stNumberInput>div>div>input {color: #4F8BF9;} </style>'''
st.markdown(colouringTool,unsafe_allow_html = True)

#Background
background = '''<style> body {
background-image: url("https://wallpaperaccess.com/full/1526027.jpg");
background-size: cover;       } </style>'''
st.markdown(background, unsafe_allow_html=True)


st.sidebar.text("Select Demo Type")

#Demo 
s = st.sidebar.selectbox((""), ["Google Forms", "Demo"])

#Text Color = 'white'
text = 'white'

max_length = 128  # Maximum length of input sentence to the model.
truncation=True
batch_size = 32
epochs = 2

# Labels in our dataset.
labels = ["contradiction", "entailment", "neutral"]


if s == "Demo":  
  class BertSemanticDataGenerator(tf.keras.utils.Sequence):
      """Generates batches of data.

      Args:
          sentence_pairs: Array of premise and hypothesis input sentences.
          labels: Array of labels.
          batch_size: Integer batch size.
          shuffle: boolean, whether to shuffle the data.
          include_targets: boolean, whether to incude the labels.

      Returns:
          Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
          (or just `[input_ids, attention_mask, `token_type_ids]`
          if `include_targets=False`)
      """

      def __init__(
          self,
          sentence_pairs,
          labels,
          batch_size=32,
          shuffle=True,
          include_targets=True,
      ):
          self.sentence_pairs = sentence_pairs
          self.labels = labels
          self.shuffle = shuffle
          self.batch_size = batch_size
          self.include_targets = include_targets
          # Load our BERT Tokenizer to encode the text.
          # We will use base-base-uncased pretrained model.
          self.tokenizer = transformers.BertTokenizer.from_pretrained(
              "bert-base-uncased", do_lower_case=True
          )
          self.indexes = np.arange(len(self.sentence_pairs))
          self.on_epoch_end()

      def __len__(self):
          # Denotes the number of batches per epoch.
          return len(self.sentence_pairs) // self.batch_size

      def __getitem__(self, idx):
          # Retrieves the batch of index.
          indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
          sentence_pairs = self.sentence_pairs[indexes]

          # With BERT tokenizer's batch_encode_plus batch of both the sentences are
          # encoded together and separated by [SEP] token.
          encoded = self.tokenizer.batch_encode_plus(
              sentence_pairs.tolist(),
              add_special_tokens=True,
              max_length=max_length,
              return_attention_mask=True,
              return_token_type_ids=True,
              pad_to_max_length=True,
              return_tensors="tf",
          )

          # Convert batch of encoded features to numpy array.
          input_ids = np.array(encoded["input_ids"], dtype="int32")
          attention_masks = np.array(encoded["attention_mask"], dtype="int32")
          token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

          # Set to true if data generator is used for training/validation.
          if self.include_targets:
              labels = np.array(self.labels[indexes], dtype="int32")
              return [input_ids, attention_masks, token_type_ids], labels
          else:
              return [input_ids, attention_masks, token_type_ids]

      def on_epoch_end(self):
          # Shuffle indexes after each epoch if shuffle is set to True.
          if self.shuffle:
              np.random.RandomState(42).shuffle(self.indexes)


              
  strategy = tf.distribute.MirroredStrategy()

  with strategy.scope():
      # Encoded token ids from BERT tokenizer.
      input_ids = tf.keras.layers.Input(
          shape=(max_length,), dtype=tf.int32, name="input_ids"
      )
      # Attention masks indicates to the model which tokens should be attended to.
      attention_masks = tf.keras.layers.Input(
          shape=(max_length,), dtype=tf.int32, name="attention_masks"
      )
      # Token type ids are binary masks identifying different sequences in the model.
      token_type_ids = tf.keras.layers.Input(
          shape=(max_length,), dtype=tf.int32, name="token_type_ids"
      )
      # Loading pretrained BERT model.
      bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
      # Freeze the BERT model to reuse the pretrained features without modifying them.
      bert_model.trainable = False

      sequence_output, pooled_output = bert_model(
          input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
      )
      # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
      bi_lstm = tf.keras.layers.Bidirectional(
          tf.keras.layers.LSTM(64, return_sequences=True)
      )(sequence_output)
      # Applying hybrid pooling approach to bi_lstm sequence output.
      avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
      max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
      concat = tf.keras.layers.concatenate([avg_pool, max_pool])
      dropout = tf.keras.layers.Dropout(0.3)(concat)
      output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
      model = tf.keras.models.Model(
          inputs=[input_ids, attention_masks, token_type_ids], outputs=output
      )

      model.compile(
          optimizer=tf.keras.optimizers.Adam(),
          loss="categorical_crossentropy",
          metrics=["acc"],
      )


  #Load the pretrained weights of the neural networks
  model.load_weights(r"/content/drive/My Drive/Project_NLP.h5")
    
  print(f"Strategy: {strategy}")


  def check_similarity(sentence1, sentence2, marks):
        sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
        test_data = BertSemanticDataGenerator(
            sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
            )

        proba = model.predict(test_data)[0]
        idx = np.argmax(proba)
        #proba = f"{proba[idx]: .2f}%"
        pred = labels[idx]
        return proba[idx]*marks



  #Entering teachers answer
  st.write('<span style="color:%s">%s</span>' % (text, "Standard Answer(Provided by the teacher) :"), unsafe_allow_html=True)
  sentence1 = st.text_input(' ')

  #Entering students answer
  st.write('<span style="color:%s">%s</span>' % (text, "Students Answer(Answer Writen by the Student) :"), unsafe_allow_html=True)
  sentence2 = st.text_input('   ')

  #Enter maximum marks.
  st.write('<span style="color:%s">%s</span>' % (text,"Maximum Marks for the question:"), unsafe_allow_html=True)
  marks =  st.number_input('         ')

  #Tokenizing the answers.
  stop_words = set(stopwords.words())
  words = word_tokenize(sentence2)

  #List storing the words excluding the stopwords.
  sentence3 = []
  for w in words:
    if w not in stop_words:
      sentence3.append(w)

  important = []
  #Keywords search
  points = 0
  st.write('<span style="color:%s">%s</span>' % (text,"Enter the important keywords: "), unsafe_allow_html=True)
  keywords = st.text_input('             ')
  words1 = word_tokenize(keywords)

  for z in words1:
    if z not in sentence3:
      points = points + 1
  #Entering marks for each keywords not mention in the answer.
  key_mark = 0.25

  #SpellChecker
  from spellchecker import SpellChecker
  spell = SpellChecker()

  #Correcting the Teacher's answers
  words = word_tokenize(sentence1)
  misspelled = spell.unknown(words)
  for i in spell.unknown(words):

    s = ("Did you mean", spell.correction(i))
    st.write('<span style="color:%s">%s</span>' % (text ,s ), unsafe_allow_html=True)
     
  # Find those words that may be misspelled
  words = word_tokenize(sentence2)
  misspelled = spell.unknown(words)
  wrong = 0
  
  for word in misspelled:
    wrong = wrong + 1
  #Spelling Correction.
  new = []
  for i in words:
    i = spell.correction(i)
    new.append(i) 
  #Evaluate the answers
  if st.button("Evaluate"):
    


 
    points1 = check_similarity(sentence1, new,marks)
    
    points1 = round(points1)
    #IF 0 spelling mistake.
    if int(wrong) == 0:
      st.write('<span style="color:%s">%s</span>' % ('Chartreuse', "There are 0 spelling mistakes"), unsafe_allow_html=True)
       
    #IF 1 spelling mistake.
    if int(wrong) == 1:
      s = ("There is 1 spelling mistake")
      st.write('<span style="color:%s">%s</span>' % ('red', s), unsafe_allow_html=True)

    #IF more than one spelling mistakes
    if int(wrong)>1:
      s  = ("There are ",wrong,"spelling mistakes")
      st.write('<span style="color:%s">%s</span>' % ('red', s), unsafe_allow_html=True)
      
    final = points1 - ((wrong)*25/100) - (points*key_mark)
    if final < 0:
      final  = 0 
    #IF marks less than 50%.
    if points1<(marks/2):
      s = ("Not much correct you got",final)
      st.write('<span style="color:%s">%s</span>' % ('red', s), unsafe_allow_html=True)
    #IF marks more than 50%
    else:
      s = (f"Good",final)
      st.write('<span style="color:%s">%s</span>' % ('Chartreuse', s), unsafe_allow_html=True) 



#Google Forms 
if s=="Google Forms":
  
 
  #Entering the marks for each question in the google form.   
  marks = 2
  number_of_questions = 2
  
  #Connecting to the answer spreadsheet.
  from google.colab import auth
  auth.authenticate_user()
  import gspread
  from oauth2client.client import GoogleCredentials

  gc = gspread.authorize(GoogleCredentials.get_application_default())
  worksheet = gc.open('Responses').sheet1
  rows = worksheet.get_all_values()
 

  # Convert to a DataFrame and evaluate.
  import pandas as pd
  df = pd.DataFrame.from_records(rows) 
  
  #Button for evaluating
  if st.button("Evaluate Google Forms"):
    
    class BertSemanticDataGenerator(tf.keras.utils.Sequence):
        """Generates batches of data.

        Args:
            sentence_pairs: Array of premise and hypothesis input sentences.
            labels: Array of labels.
            batch_size: Integer batch size.
            shuffle: boolean, whether to shuffle the data.
            include_targets: boolean, whether to incude the labels.

        Returns:
            Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
            (or just `[input_ids, attention_mask, `token_type_ids]`
            if `include_targets=False`)
        """

        def __init__(
            self,
            sentence_pairs,
            labels,
            batch_size=32,
            shuffle=True,
            include_targets=True,
        ):
            self.sentence_pairs = sentence_pairs
            self.labels = labels
            self.shuffle = shuffle
            self.batch_size = batch_size
            self.include_targets = include_targets
            # Load our BERT Tokenizer to encode the text.
            # We will use base-base-uncased pretrained model.
            self.tokenizer = transformers.BertTokenizer.from_pretrained(
                "bert-base-uncased", do_lower_case=True
            )
            self.indexes = np.arange(len(self.sentence_pairs))
            self.on_epoch_end()

        def __len__(self):
            # Denotes the number of batches per epoch.
            return len(self.sentence_pairs) // self.batch_size

        def __getitem__(self, idx):
            # Retrieves the batch of index.
            indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
            sentence_pairs = self.sentence_pairs[indexes]

            # With BERT tokenizer's batch_encode_plus batch of both the sentences are
            # encoded together and separated by [SEP] token.
            encoded = self.tokenizer.batch_encode_plus(
                sentence_pairs.tolist(),
                add_special_tokens=True,
                max_length=max_length,
                return_attention_mask=True,
                return_token_type_ids=True,
                pad_to_max_length=True,
                return_tensors="tf",
            )

            # Convert batch of encoded features to numpy array.
            input_ids = np.array(encoded["input_ids"], dtype="int32")
            attention_masks = np.array(encoded["attention_mask"], dtype="int32")
            token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

            # Set to true if data generator is used for training/validation.
            if self.include_targets:
                labels = np.array(self.labels[indexes], dtype="int32")
                return [input_ids, attention_masks, token_type_ids], labels
            else:
                return [input_ids, attention_masks, token_type_ids]

        def on_epoch_end(self):
            # Shuffle indexes after each epoch if shuffle is set to True.
            if self.shuffle:
                np.random.RandomState(42).shuffle(self.indexes)


                
    strategy = tf.distribute.MirroredStrategy()

    with strategy.scope():
        # Encoded token ids from BERT tokenizer.
        input_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="input_ids"
        )
        # Attention masks indicates to the model which tokens should be attended to.
        attention_masks = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="attention_masks"
        )
        # Token type ids are binary masks identifying different sequences in the model.
        token_type_ids = tf.keras.layers.Input(
            shape=(max_length,), dtype=tf.int32, name="token_type_ids"
        )
        # Loading pretrained BERT model.
        bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
        # Freeze the BERT model to reuse the pretrained features without modifying them.
        bert_model.trainable = False

        sequence_output, pooled_output = bert_model(
            input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
        )
        # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
        bi_lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(64, return_sequences=True)
        )(sequence_output)
        # Applying hybrid pooling approach to bi_lstm sequence output.
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
        concat = tf.keras.layers.concatenate([avg_pool, max_pool])
        dropout = tf.keras.layers.Dropout(0.3)(concat)
        output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
        model = tf.keras.models.Model(
            inputs=[input_ids, attention_masks, token_type_ids], outputs=output
        )

        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="categorical_crossentropy",
            metrics=["acc"],
        )


    #Load the pretrained weights of the neural networks
    model.load_weights(r"/content/drive/My Drive/Project_NLP.h5")
      
    print(f"Strategy: {strategy}")


    def check_similarity(sentence1, sentence2, marks):
          sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
          test_data = BertSemanticDataGenerator(
              sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
              )

          proba = model.predict(test_data)[0]
          idx = np.argmax(proba)
          #proba = f"{proba[idx]: .2f}%"
          pred = labels[idx]
          return proba[idx]*marks


     
    points = 0
    #Open the google spreadsheet.
    worksheet = gc.open('Responses').sheet1
    data = worksheet.get_all_records()
    num_of_rows = len(data)

    
    num_of_cols = number_of_questions+1
    l = []
    h = 1
    avg_val = 0
    avg_marks = []
  
    
    while h < num_of_cols:
      h = h + 1
      #Iterate through all the columns 
      cols = worksheet.col_values(h)
      for j in cols: 
        l.append(j)
      #This is answer for the first question
      sentence1 = l[1]   
      p = 2
      #Iterate through all the rows in the columns.
      while p <= num_of_rows:
        p = p + 1
        sentence2 = worksheet.cell(p,h).value
        #Tokenize the sentence      
        stop_words = set(stopwords.words())
        words = word_tokenize(sentence2)
        sentence3 = []
        for w in words:
          if w not in stop_words:
            sentence3.append(w)

        from spellchecker import SpellChecker
        spell = SpellChecker()
        #Correcting the Teacher's answers
        words = word_tokenize(sentence1)
        misspelled = spell.unknown(words)
        i = 0    
        #Find those words that may be misspelled
        words = word_tokenize(sentence2)
        misspelled = spell.unknown(words)
        wrong = 0       
        for word in misspelled:
            wrong = wrong + 1
        new = []
        for i in words:
          i = spell.correction(i)
          new.append(i)
          

        points1 = check_similarity(sentence1,sentence2,marks)
        points1 = round(points1)
        #Cut marks for spelling mistakes
        final = points1 - ((wrong)*25/100)
        st.write('<span style="color:%s">%s</span>' % ('Chartreuse',sentence2), unsafe_allow_html=True) 
        st.write('<span style="color:%s">%s</span>' % ('white', final/marks), unsafe_allow_html=True)
        s = ('Spellng you got wrong', misspelled)
        st.write('<span style="color:%s">%s</span>' % ('red',s), unsafe_allow_html=True) 

      #   avg_val = avg_val + (final/marks*100)
      # avg = (avg_val/num_of_rows)
      # avg_marks.append(avg)
      # avg=0
           
    # #Plotting the graphs
    # st.write('<span style="color:%s">%s</span>' % (text ,"Average Marks" ), unsafe_allow_html=True)
    # import matplotlib.pyplot as plt
    # fig = plt.figure()
    # ax = fig.add_axes([0,0,1,1])
    # langs = ['Question1', 'Question2']
    # ax.bar(langs,avg_marks)
    # st.pyplot(fig)  

Writing main.py


In [4]:
localhost:8501

In [5]:
from pyngrok import ngrok
ngrok.kill()
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
print(public_url)     

http://cacbf9519a93.ngrok.io


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Support for "return_ngrok_tunnel" as "False" is deprecated and will be removed in 5.0.0, when this method will return a NgrokTunnel instead of a str
  after removing the cwd from sys.path.


In [6]:
!streamlit run main.py                    

2021-01-21 09:51:36.338349: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://104.199.138.213:8501

2021-01-21 09:51:39.426 Refreshing access_token
  Stopping...
